In [1]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

c:\Users\faust\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Input Problem parameters

In [2]:
import json

l=["small", "medium", "large"]
res = dict()

for taille in l:
    nom_fichier = taille+".json"
    fichier = open(nom_fichier)
    print("I opened ", nom_fichier, " filed. \n")

    data = json.load(fichier)
    print("I loaded ", nom_fichier, " data in a dict. \n")

    res[taille] = data
    print("I stored ", nom_fichier, " data in res. \n")

    fichier.close()

I opened  small.json  filed. 

I loaded  small.json  data in a dict. 

I stored  small.json  data in res. 

I opened  medium.json  filed. 

I loaded  medium.json  data in a dict. 

I stored  medium.json  data in res. 

I opened  large.json  filed. 

I loaded  large.json  data in a dict. 

I stored  large.json  data in res. 



In [3]:
res["small"]

{'horizon': 5,
 'qualifications': ['A', 'B', 'C'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': []},
  {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
  {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}],
 'jobs': [{'name': 'Job1',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
  {'name': 'Job2',
   'gain': 15,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 2}},
  {'name': 'Job3',
   'gain': 15,
   'due_date': 4,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'C': 2}},
  {'name': 'Job4',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'B': 2, 'C': 1}},
  {'name': 'Job5',
   'gain': 10,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'C': 2}}]}

In [4]:
res["medium"]

{'horizon': 22,
 'qualifications': ['F', 'G', 'A', 'D', 'J', 'C', 'H', 'E', 'B', 'I'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': [1, 2]},
  {'name': 'Liam', 'qualifications': ['A', 'D', 'E'], 'vacations': [1, 2]},
  {'name': 'Emma', 'qualifications': ['B', 'H'], 'vacations': [8, 9]},
  {'name': 'Noah',
   'qualifications': ['G', 'D', 'J', 'C', 'H', 'I'],
   'vacations': []},
  {'name': 'Amelia',
   'qualifications': ['F', 'G', 'J', 'E'],
   'vacations': [16, 15]}],
 'jobs': [{'name': 'Job1',
   'gain': 15,
   'due_date': 20,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 4}},
  {'name': 'Job2',
   'gain': 30,
   'due_date': 16,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 2, 'C': 1, 'D': 4}},
  {'name': 'Job3',
   'gain': 30,
   'due_date': 12,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'C': 1}},
  {'name': 'Job4',
   'gain': 30,
   'due_date': 18,
   'daily_penal

In [5]:
res["large"]

{'horizon': 36,
 'qualifications': ['E', 'I', 'J', 'A', 'C', 'H', 'G', 'D', 'B', 'F'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['B', 'A', 'C'],
   'vacations': [1, 2, 3, 4, 5]},
  {'name': 'Liam',
   'qualifications': ['E', 'A', 'D'],
   'vacations': [6, 7, 8, 9, 10]},
  {'name': 'Emma',
   'qualifications': ['B', 'H'],
   'vacations': [21, 22, 23, 24, 25]},
  {'name': 'Noah',
   'qualifications': ['C', 'H', 'G', 'D'],
   'vacations': [26, 27, 28, 29, 30]},
  {'name': 'Amelia',
   'qualifications': ['E', 'I', 'G', 'J', 'F'],
   'vacations': []},
  {'name': 'Oliver',
   'qualifications': ['J', 'F', 'G'],
   'vacations': [32, 33, 31]}],
 'jobs': [{'name': 'Job1',
   'gain': 15,
   'due_date': 26,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 4}},
  {'name': 'Job2',
   'gain': 30,
   'due_date': 22,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 4, 'B': 2, 'C': 1, 'D': 4}},
  {'name': 'Job3',
   'gain': 30,
   'due_date': 16,
   'd

In [6]:
l

['small', 'medium', 'large']

In [ ]:
## à changer selon le fichier que l'on veut
## traiter
taille = "small"

In [ ]:
horizon = res[taille]["horizon"]
qualifications = res[taille]["qualifications"]
staff = res[taille]["staff"]
jobs = res[taille]["jobs"]

In [ ]:
nombre_projets = len(jobs)
nombre_qualif = len(qualifications)
nombre_employes = len(staff)

Définissons les variables

In [ ]:
## besoin_i_j, le projet j a besoin de la qualif i
besoin = [[0]*nombre_projets]*nombre_qualif

## conge_k_n, l'employé k est en congés au jours n
conge = [[0]*horizon]*nombre_employes

## qualif_i_k, l'employé k a la qualif i
qualif = [[0]*nombre_employes]*nombre_qualif

## d_j, date de livraison selon les projets
d = [0]*nombre_projets

## p_j, pénalités selon les projets
p = [0]*nombre_projets

## b_j, bénéfices selon les projets
b = [0]*nombre_projets

Définissons les paramètres

In [ ]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x = [[[[0]*horizon]*nombre_employes]*nombre_projets]*nombre_qualif


In [ ]:
retailers, deliveryPoints, spiritMarket = gp.multidict({
    (1): [11,34],
    (2): [47,411],
    (3): [44,82],
    (4): [25,157],
    (5): [10,5],
    (6): [26,183],
    (7): [26,14],
    (8): [54,215],
    (9): [18,102],
    (10): [51,21],
    (11): [20,54],
    (12): [105,0],
    (13): [7,6],
    (14): [16,96],
    (15): [34,118],
    (16): [100,112],
    (17): [50,535],
    (18): [21,8],
    (19): [11,53],
    (20): [19,28],
    (21): [14,69],
    (22): [10,65],
    (23): [11,27]
})

# Create a dictionary to capture the oil market -in millions of gallons for region 1.

retailers1,  oilMarket1 = gp.multidict({
    (1): 9,
    (2): 13,
    (3): 14,
    (4): 17,
    (5): 18,
    (6): 19,
    (7): 23,
    (8): 21
})

# Create a dictionary to capture the oil market -in millions of gallons for region 2.

retailers2,  oilMarket2 = gp.multidict({
    (9): 9,
    (10): 11,
    (11): 17,
    (12): 18,
    (13): 18,
    (14): 17,
    (15): 22,
    (16): 24,
    (17): 36,
    (18): 43
})

# Create a dictionary to capture the oil market -in millions of gallons for region 3.

retailers3,  oilMarket3 = gp.multidict({
    (19): 6,
    (20): 15,
    (21): 15,
    (22): 25,
    (23): 39
})

# Create a dictionary to capture retailers in group A.

groupA,  retailerA = gp.multidict({
    (1): 1,
    (2): 1,
    (3): 1,
    (5): 1,
    (6): 1,
    (10): 1,
    (15): 1,
    (20): 1
})

# Create a dictionary to capture retailers in group B.

groupB,  retailerB = gp.multidict({
    (4): 1,
    (7): 1,
    (8): 1,
    (9): 1,
    (11): 1,
    (12): 1,
    (13): 1,
    (14): 1,
    (16): 1,
    (17): 1,
    (18): 1,
    (19): 1,
    (21): 1,
    (22): 1,
    (23): 1
})

# Forty and five percentages of each goal

deliveryPoints40 = 292
deliveryPoints5 = 36.5
spiritMarket40 = 958
spiritMarket5 = 119.75
oilMarket1_40 = 53.6
oilMarket1_5 = 6.7
oilMarket2_40 = 86
oilMarket2_5 = 10.75
oilMarket3_40 = 40
oilMarket3_5 = 5
retailerA40 = 3.2
retailerA5 = 0.4
retailerB40 = 6
retailerB5 = 0.75